In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
!make -C csrc/bcc/libbpf-tools

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'


In [3]:
!make -C csrc/bpftool/src

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'
...                        libbfd: [ on  ]
...               clang-bpf-co-re: [ on  ]
...                          llvm: [ on  ]
...                        libcap: [ OFF ]
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'


In [34]:
from depsurf import BCC_OBJ_PATH, BPFObject
from typing import List

assert BCC_OBJ_PATH.exists(), f"Path not found: {BCC_OBJ_PATH}"

result: List[BPFObject] = []

for path in sorted(BCC_OBJ_PATH.glob("*.bpf.o")):
    obj_file = BPFObject(path)
    obj_file.deps  # trigger the calculation of dependencies
    result.append(obj_file)

result[1].deps

[decorator.py:17 ] INFO: gen_min_btf      Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf
[decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.json
[decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bashreadline.bpf.min.btf.txt
[decorator.py:17 ] INFO: gen_min_btf      Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf
[decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf.json
[decorator.py:17 ] INFO: dump_btf_impl    Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/bindsnoop.bpf.min.btf.txt
[decorator.py:17 ] INFO: gen_min_btf      Using /Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools/.output/biolatency.bpf.min.btf
[decorator.py:17 ] INFO: dump_btf_

[('Function', 'inet6_bind'),
 ('Function', 'inet_bind'),
 ('Field', 'in6_addr::in6_u'),
 ('Field', 'inet_sock::bind_address_no_port'),
 ('Field', 'inet_sock::freebind'),
 ('Field', 'inet_sock::inet_flags'),
 ('Field', 'inet_sock::inet_saddr'),
 ('Field', 'inet_sock::inet_sport'),
 ('Field', 'inet_sock::transparent'),
 ('Field', 'sock::__sk_common'),
 ('Field', 'sock::sk_protocol'),
 ('Field', 'sock_common::skc_bound_dev_if'),
 ('Field', 'sock_common::skc_reuse'),
 ('Field', 'sock_common::skc_reuseport'),
 ('Field', 'sock_common::skc_v6_rcv_saddr'),
 ('Field', 'socket::sk'),
 ('Field', 'user_pt_regs::regs')]

In [5]:
from depsurf import BuildVersion

versions = BuildVersion.filter()
for v in versions:
    v.img.btf  # trigger loading

[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.4.0-21-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.8.0-22-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.10.0-19-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.13.0-16-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.15.0-20-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/4.18.0-10-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.0.0-13-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.3.0-18-generic-amd64.pkl
[      btf.py:19 ] INFO: Loading BTF from /Users/szhong/Down

In [116]:
import pandas as pd
from pathlib import Path
from depsurf import OUTPUT_PATH, DepKind
from depsurf.plot import save_df


class Report:
    def __init__(self, obj_file: "BPFObject", versions: list[BuildVersion], path: Path):
        self.obj_file = obj_file
        self.versions = versions
        self.file = open(path, "w")

    def __del__(self):
        self.file.close()

    @staticmethod
    def deps_filter(kind, name) -> bool:
        if kind in (DepKind.STRUCT_FIELD, DepKind.STRUCT):
            return not name.startswith("user_pt_regs")

        if kind == DepKind.FUNC:
            return not (name.startswith("dummy") or name == "foo")

        if kind in (DepKind.TRACEPOINT, DepKind.LSM):
            return True

        return False

    def report(self):
        import pandas as pd

        result_version = {}
        result_delta = {}

        for kind, name in self.obj_file.deps:
            if not Report.deps_filter(kind, name):
                continue
            print(f"{kind:12}{name}", file=self.file)
            key = (self.obj_file.name, kind, name)
            result_version[key], result_delta[key] = self.report_dep(kind, name)

        df_version = pd.DataFrame(result_version).T
        df_delta = pd.DataFrame(result_delta).T

        return df_version, df_delta

    def report_dep(self, kind: DepKind, name: str):
        result_version = {}
        result_delta = {}

        for v in self.versions:
            result_version[v.short_version] = kind.status(v.img, name)

        version_str  = ""
        for status in result_version.values():
            version_str += "✅" if status else "❌"
        print(f"\tAvailable versions: {version_str}", file=self.file)

        for v1, v2 in zip(self.versions, self.versions[1:]):
            changes = kind.diff_kind(v1.img, v2.img, name)
            if changes is None:
                num = -1
            elif not changes:
                num = 0
            else:
                num = len(changes)
                print(f"\t{v1.short_version} -> {v2.short_version}: {num} changes", file=self.file)
                for change in changes:
                    print(f"\t\t{change}", file=self.file)
            result_delta[(v1.short_version, v2.short_version)] = num

        return result_version, result_delta

result_version = []
result_delta = []
for obj_file in result:
    result_path = OUTPUT_PATH / "bcc" / f"{obj_file.name}.log"
    report = Report(obj_file, versions, result_path)
    cs, cd = report.report()
    result_version.append(cs)
    result_delta.append(cd)


df_delta = pd.concat(result_delta)
df_delta.index.names = ["Program", "Type", "Name"]
save_df(df_delta, "bcc_delta")


df_version = pd.concat(result_version)
df_version.index.names = ["Program", "Type", "Name"]
save_df(df_version, "bcc_version")

print("Done")

[       pd.py:30 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/bcc_delta.pkl
[       pd.py:34 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/bcc_delta.txt
[       pd.py:30 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/bcc_version.pkl
[       pd.py:34 ] INFO: Saved dataframe to /Users/szhong/Downloads/bpf-study/output/bcc_version.txt


Done


In [114]:
from depsurf.plot import load_df

df_delta = load_df("bcc_delta")
df_delta = df_delta.replace({0: ".", -1: ""})
df_delta


[       pd.py:45 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/bcc_delta.pkl


4.4  4.8 4.10 4.13 4.15 4.18 5.0 5.3 5.4  5.8 5.11 5.13 5.15 5.19 6.2 6.5
                                                                        4.8 4.10 4.13 4.15 4.18  5.0 5.3 5.4 5.8 5.11 5.13 5.15 5.19  6.2 6.5 6.8
Program     Type       Name                                                                                                                      
bindsnoop   Function   inet6_bind                                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_bind                                          .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
            Field      in6_addr::in6_u                                    .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock::bind_address_no_port                    .    .    .    .    .    .   .   .   .    .    .    .    .    .   .    
                       inet_sock::freebind                                .    .    .    .    .    .   .   .   .    .    .    .    .    .   .    
                       inet_sock::inet_flags                                                                                                     
                       inet_sock::inet_saddr                              .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock::inet_sport                              .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       inet_sock::transparent                             .    .    .    .    .    .   .   .   .    .    .    .    .    .   .    
                       sock::__sk_common                                  .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock::sk_protocol                                  .    .    .    .    .    .   .   .   1    .    .    .    .    .   .   .
                       sock_common::skc_bound_dev_if                      .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock_common::skc_reuse                             .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock_common::skc_reuseport                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       sock_common::skc_v6_rcv_saddr                      .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       socket::sk                                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
biolatency  Tracepoint block_rq_complete                                  .    .    5    .    .    .   .   .   .    .    .    .    1    .   .   .
                       block_rq_insert                                    .    .    .    .    .    .   .   .   .    2    .    .    .    .   .   .
                       block_rq_issue                                     .    .    .    .    .    .   .   .   .    2    .    .    .    .   .   .
            Field      gendisk::first_minor                               .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       gendisk::major                                     .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       request::cmd_flags                                 .    1    .    .    .    .   .   .   .    .    .    .    .    1   .   .
                       request::q                                         .    .    .    .    .    .   .   .   .    .    .    .    .    .   .   .
                       request::rq_disk                                   .    .    .    .    .    .   .   .   .    .    .    .                  
                       request_queue::disk                                                                                        

In [97]:
from depsurf.plot import load_df

df_version = load_df("bcc_version")
df_version = df_version.replace({True: "✅", False: "❌"})
df_version

[       pd.py:45 ] INFO: Loaded df from /Users/szhong/Downloads/bpf-study/output/bcc_version.pkl


4.4 4.8 4.10 4.13 4.15 4.18 5.0 5.3 5.4 5.8 5.11 5.13 5.15 5.19 6.2 6.5 6.8
Program     Type       Name                                                                                                                        
bindsnoop   Function   inet6_bind                                         ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       inet_bind                                          ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
            Field      in6_addr::in6_u                                    ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       inet_sock::bind_address_no_port                    ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ❌
                       inet_sock::freebind                                ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ❌
                       inet_sock::inet_flags                              ❌   ❌    ❌    ❌    ❌    ❌   ❌   ❌   ❌   ❌    ❌    ❌    ❌    ❌   ❌   ❌   ✅
                       inet_sock::inet_saddr                              ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       inet_sock::inet_sport                              ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       inet_sock::transparent                             ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ❌
                       sock::__sk_common                                  ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       sock::sk_protocol                                  ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       sock_common::skc_bound_dev_if                      ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       sock_common::skc_reuse                             ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       sock_common::skc_reuseport                         ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       sock_common::skc_v6_rcv_saddr                      ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       socket::sk                                         ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
biolatency  Tracepoint block_rq_complete                                  ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       block_rq_insert                                    ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       block_rq_issue                                     ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
            Field      gendisk::first_minor                               ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       gendisk::major                                     ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       request::cmd_flags                                 ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       request::q                                         ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅
                       request::rq_disk                                   ✅   ✅    ✅    ✅    ✅    ✅   ✅   ✅   ✅   ✅    ✅    ✅    ✅    ❌   ❌   ❌   ❌
                       request_queue::disk                                ❌   ❌    ❌    ❌    ❌    ❌   ❌   ❌   ❌   ❌    ❌    ❌    ✅    ✅   ✅   ✅   ✅
                       request_queue::elevator                            ✅ 